# Δεδομένα Πειράματος και Διαλογή Εικόνων
## Με αυτό το αρχείο παρουσιάζουμε σε μορφή gif ποιες εικόνες από το πείραμα θα χρησιμοποιηθούν 
1. Οι εικόνες βρίσκονται στο dir `/data/raw`
2. το notebook `01-data_access.ipynb` βρίσκεται στο `/notebooks`
3. οπότε για να έχουμε πρόσβαση πρέπει να μεταφερθούμε στο root project directory 

In [1]:
# εντολή bash και γενικά η πρώτη εντολή που πρέπει να εκτελέσει κάποιος όταν ανοίγει αυτό το αρχείο 
!cd ..

In [3]:
from skimage.io import imread

In [5]:
image = imread(fname='star-trail.jpg')

array([[[ 5,  8, 15],
        [ 5,  8, 15],
        [ 5,  8, 15],
        ...,
        [28, 30, 42],
        [27, 29, 41],
        [26, 28, 40]],

       [[ 7,  7, 15],
        [ 7,  7, 15],
        [ 7,  7, 15],
        ...,
        [30, 32, 44],
        [27, 29, 41],
        [24, 26, 38]],

       [[ 7,  7, 15],
        [ 7,  7, 15],
        [ 7,  7, 15],
        ...,
        [29, 31, 43],
        [26, 28, 40],
        [24, 26, 38]],

       ...,

       [[11, 15, 24],
        [10, 13, 22],
        [ 7, 10, 19],
        ...,
        [41, 43, 66],
        [39, 41, 64],
        [36, 39, 58]],

       [[ 9, 13, 22],
        [10, 13, 22],
        [ 8, 11, 20],
        ...,
        [41, 43, 64],
        [37, 39, 60],
        [33, 36, 53]],

       [[ 7, 11, 20],
        [ 9, 12, 21],
        [ 9, 12, 21],
        ...,
        [40, 43, 62],
        [34, 37, 56],
        [28, 31, 46]]], dtype=uint8)

In [6]:
import matplotlib as plt
plt.imshow